In [9]:
# %tensorflow_version 2.x
import tensorflow as tf
import numpy as np
import cv2,glob,os

In [10]:
class_list = ['Normal','Corona','BacterialPneumonia']

In [11]:
train_dataset = tf.data.TFRecordDataset(glob.glob('/content/drive/My Drive/CoronaVirusData/TrainData/*.tfrecord'),num_parallel_reads=20)
val_dataset = tf.data.TFRecordDataset('/content/drive/My Drive/CoronaVirusData/Validation.tfrecord')
test_dataset = tf.data.TFRecordDataset('/content/drive/My Drive/CoronaVirusData/Test.tfrecord')

# Create a dictionary describing the features.
image_feature_description = {
    'image_raw': tf.io.FixedLenFeature([], tf.string),
    'label': tf.io.FixedLenFeature([], tf.int64)
}

def _parse_image_function(example_proto):
  features = tf.io.parse_single_example(example_proto, image_feature_description)
  image_raw = tf.image.decode_jpeg(features['image_raw'],channels=1)
  image_raw = tf.cast(image_raw , tf.float32) * (1. / 255)
  image_raw = tf.image.resize(image_raw,(300,400))

  label = features['label']

  return image_raw,label

train_dataset = train_dataset.map(_parse_image_function)
val_dataset = val_dataset.map(_parse_image_function)
test_dataset = test_dataset.map(_parse_image_function)

In [0]:
train_dataset = train_dataset.repeat().shuffle(600).batch(16).prefetch(8)
val_dataset = val_dataset.repeat().shuffle(10).batch(5)

In [23]:
# manually finding out the class weights to be assigned since I dont know any other way for tensors

samples = {(339+279+54+63)/(3*339),(339+279+54+63)/(3*279),(339+279+54+63)/(3*(54+63))}
print(samples)

{0.7227138643067846, 0.8781362007168458, 2.094017094017094}


In [4]:
from tensorflow.keras.layers import Dense,Conv2D,SeparableConv2D,Dropout,MaxPool2D,MaxPooling2D,GlobalMaxPooling2D,BatchNormalization,Flatten,Dense

def create_model(shape=(300,400,1)):
    model = tf.keras.models.Sequential()
    model.add(Conv2D(32,(5,5),input_shape=shape))
    model.add(MaxPooling2D())
    model.add(BatchNormalization())
    model.add(Conv2D(16,(3,3)))
    model.add(MaxPooling2D())
    model.add(BatchNormalization())
    model.add(Conv2D(8,(2,2)))
    model.add(MaxPooling2D())
    model.add(BatchNormalization())    
    model.add(Flatten())
    model.add(Dense(16, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(8, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(len(class_list),activation='softmax'))          

    return model

In [12]:
model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 296, 396, 32)      832       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 148, 198, 32)      0         
_________________________________________________________________
batch_normalization (BatchNo (None, 148, 198, 32)      128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 146, 196, 16)      4624      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 73, 98, 16)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 73, 98, 16)        64        
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 97, 8)         5

In [0]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(loss='sparse_categorical_crossentropy',optimizer=optimizer,metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

In [0]:
tf.keras.backend.clear_session()

In [0]:
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2,verbose=1)
model_checkpoint = tf.keras.callbacks.ModelCheckpoint('Checkpoint1.h5',save_best_only=True,verbose=1)

In [107]:
model.fit(train_dataset,validation_data=val_dataset,shuffle=True,epochs=10,class_weight=samples,steps_per_epoch=(672+63)//16,validation_steps=75//5,callbacks=[early_stopping,model_checkpoint])

Train for 45 steps, validate for 15 steps
Epoch 1/10
44/45 [============================>.] - ETA: 0s - loss: 0.5003 - sparse_categorical_accuracy: 0.8381
Epoch 00001: val_loss improved from 0.43569 to 0.38108, saving model to Checkpoint1.h5
45/45 [==============================] - 13s 298ms/step - loss: 0.4994 - sparse_categorical_accuracy: 0.8375 - val_loss: 0.3811 - val_sparse_categorical_accuracy: 0.9600
Epoch 2/10
44/45 [============================>.] - ETA: 0s - loss: 0.4747 - sparse_categorical_accuracy: 0.8210
Epoch 00002: val_loss did not improve from 0.38108
45/45 [==============================] - 7s 150ms/step - loss: 0.4740 - sparse_categorical_accuracy: 0.8236 - val_loss: 0.6113 - val_sparse_categorical_accuracy: 0.7733
Epoch 3/10
44/45 [============================>.] - ETA: 0s - loss: 0.4515 - sparse_categorical_accuracy: 0.8253
Epoch 00003: val_loss improved from 0.38108 to 0.37149, saving model to Checkpoint1.h5
45/45 [==============================] - 7s 159ms/step 

In [0]:
model = tf.keras.models.load_model('Checkpoint1.h5')

In [108]:
preds=model.predict(test_dataset.batch(15),batch_size=None)
preds = [np.argmax(i) for i in preds]
preds

[1, 1, 1, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2]

In [109]:
predictions = []
for i,j in test_dataset:
  predictions.append(j.numpy())
predictions  

[1, 1, 1, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2]

In [110]:
count=0
for i,j in zip(preds,predictions):
  if i==j:
    count+=1
print(count/len(preds))    

1.0


In [0]:
model.save('/content/drive/My Drive/CoronaVirusData/COVID-19-Detector.h5')